In [33]:
import pandas as pd
import sqlite3

In [34]:
#Conectar con la base de datos existente
conn = sqlite3.connect('../sql/supply_chain.db')

In [35]:
#Consultar las columnas de la tabla (Consultar columnas, tipo de datos, nulos y pk)
query = "PRAGMA table_info(inventario);"
pd.read_sql_query(query, conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Product type,TEXT,0,None,0
1,1,SKU,TEXT,0,None,0
2,2,Price,REAL,0,None,0
3,3,Availability,INTEGER,0,None,0
4,4,Number of products sold,INTEGER,0,None,0
5,5,Revenue generated,REAL,0,None,0
6,6,Customer demographics,TEXT,0,None,0
7,7,Stock levels,INTEGER,0,None,0
8,8,Lead times,INTEGER,0,None,0
9,9,Order quantities,INTEGER,0,None,0


In [36]:
#Consultar las primeras 10 filas de datos
query = "SELECT * FROM inventario LIMIT 10;"
pd.read_sql_query(query, conn)

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632
5,haircare,SKU5,1.699976,87,147,2828.348746,Non-binary,90,27,66,...,Bangalore,10,104,17,56.766476,Fail,2.779194,Road,Route A,235.461237
6,skincare,SKU6,4.078333,48,65,7823.476560,Male,11,15,58,...,Kolkata,14,314,24,1.085069,Pending,1.000911,Sea,Route A,134.369097
7,cosmetics,SKU7,42.958384,59,426,8496.103813,Female,93,17,11,...,Bangalore,22,564,1,99.466109,Fail,0.398177,Road,Route C,802.056312
8,cosmetics,SKU8,68.717597,78,150,7517.363211,Female,5,10,15,...,Mumbai,13,769,8,11.423027,Pending,2.709863,Sea,Route B,505.557134
9,skincare,SKU9,64.015733,35,980,4971.145988,Unknown,14,27,83,...,Chennai,29,963,23,47.957602,Pending,3.844614,Rail,Route B,995.929461


In [37]:
#Consultar la cantidad de filas
query = "SELECT COUNT(*) FROM inventario"
pd.read_sql_query(query,conn)

,COUNT(*)
0,100


In [38]:
#Consultar valores únicos de tipo de produto
query = "SELECT DISTINCT [Product type] FROM inventario"
pd.read_sql_query(query, conn)

,Product type
0,haircare
1,skincare
2,cosmetics


### 1. Consulta de Stock


#### 1.1 Rotation (new KPI)
En esta análise consulto el total de estoque por produto y por categoria y creo una KPI con la rotation por produto y la rotación média y exporto a un csv limpio.

In [39]:
#Contar valores por tipo de producto
query = '''
SELECT [Product type], COUNT(*) AS total_productos
FROM inventario
GROUP BY [Product type]
ORDER BY total_productos DESC
'''
pd.read_sql_query(query,conn)

,Product type,total_productos
0,skincare,40
1,haircare,34
2,cosmetics,26


In [40]:
#Calcular rotation a partir de la divison de productos vendidos por level de estoque
query = '''
SELECT 
    SKU,
    [Number of products sold], 
    [Stock levels],
    [Number of products sold] / [Stock levels] AS 'rotation'
FROM inventario
'''
pd.read_sql_query(query,conn)

,SKU,Number of products sold,Stock levels,rotation
0,SKU0,802,58,13.0
1,SKU1,736,53,13.0
2,SKU2,8,1,8.0
3,SKU3,83,23,3.0
4,SKU4,871,5,174.0
...,...,...,...,...
95,SKU95,672,15,44.0
96,SKU96,324,67,4.0
97,SKU97,62,46,1.0
98,SKU98,913,53,17.0


In [41]:
#Crear una KPI nueva que muestre la rotacion media de cada tipo de producto
query = '''
SELECT
    [Product type],
    SUM([Number of products sold]) AS total_sold, 
    SUM([Stock levels]) AS stock_total,
    SUM([Number of products sold]) * 1.0 / SUM([Stock levels]) AS rotation_media
FROM inventario
GROUP BY [Product type]
'''
#Ejecutar la query y guardar el resultado en un DataFrame.
df_rotation = pd.read_sql_query(query,conn)

#Exportar el Df a un csv limpio para abrir en powerBi
df_rotation.to_csv('../data/processed/rotacion_por_categoria.csv', index=False)

#Cerrar la conexión con el servidor
#conn.close()

#### 1.2 Analise de Ventas y stock a partir de KPI rotation

In [43]:
#Consulta de ventas y stock por produtos
query = ''' 
SELECT 
    SKU,
    [Product type],
    [Number of products sold],
    [Stock levels]
FROM inventario
ORDER BY [Stock levels] DESC
'''
pd.read_sql_query(query,conn)




,SKU,Product type,Number of products sold,Stock levels
0,SKU12,haircare,336,100
1,SKU51,haircare,154,100
2,SKU59,cosmetics,484,100
3,SKU91,cosmetics,916,98
4,SKU49,cosmetics,99,97
...,...,...,...,...
95,SKU47,skincare,910,4
96,SKU16,skincare,280,2
97,SKU2,haircare,8,1
98,SKU34,skincare,602,1


In [49]:
query = '''
SELECT 
    SKU,
    [Product type],
    [Number of products sold],
    [Stock levels],
    ROUND(1.0 * [Number of products sold] / [Stock levels], 2) AS rotation
FROM inventario
WHERE rotation < 2
ORDER BY rotation ASC
LIMIT 30
'''
pd.read_sql_query(query, conn)

,SKU,Product type,Number of products sold,Stock levels,rotation
0,SKU45,haircare,24,93,0.26
1,SKU48,haircare,29,30,0.97
2,SKU49,cosmetics,99,97,1.02
3,SKU97,haircare,62,46,1.35
4,SKU89,cosmetics,134,90,1.49
5,SKU51,haircare,154,100,1.54
6,SKU5,haircare,147,90,1.63
7,SKU85,cosmetics,25,15,1.67
8,SKU25,haircare,142,82,1.73
9,SKU93,haircare,114,63,1.81


In [50]:
query= '''
SELECT 
    [Product type],
    COUNT(*) AS cantidad_productos,
    AVG(ROUND(1.0 * [Number of products sold] / [Stock levels], 2)) AS rotacion_promedio
FROM inventario
WHERE (1.0 * [Number of products sold] / [Stock levels]) < 2
GROUP BY [Product type]
ORDER BY cantidad_productos DESC
'''
pd.read_sql_query(query, conn)

,Product type,cantidad_productos,rotacion_promedio
0,haircare,7,1.327143
1,cosmetics,3,1.393333
2,skincare,1,1.840000


#### 1.3 Impacto financeiro por produto parado en stock

In [61]:
#Valor total de todos los productos en stock
query = ''' 
SELECT 
    ROUND(SUM([Stock levels] * Price), 2) AS valor_total_stock
FROM inventario
'''

#Expotacion del Valor total de todo el stock a un csv limpio 
df1 = pd.read_sql_query(query, conn)
df1.to_csv('../data/processed/valor_total_stock.csv', index=False)

In [62]:
# Valor del stock inmovilizado (productos con rotación baja)
query = '''
SELECT 
    ROUND(SUM([Stock levels] * Price), 2) AS valor_stock_inmovilizado
FROM inventario
WHERE (1.0 * [Number of products sold] / [Stock levels]) < 2
'''
#Expotacion del Valor del stock inmovilizado a un csv limpio 
df2 = pd.read_sql_query(query, conn)
df2.to_csv('../data/processed/valor_inmovilizado.csv', index=False)

In [ ]:
#Valor inmovilizado por stock con rotación abajo de 2 por categoria
query = '''
SELECT
    [Product type],
    COUNT(*) AS productos_inmovilizados,
    ROUND(SUM([Stock levels] * Price), 2) AS valor_inmovilizados_categoria,
    ROUND(SUM([Stock levels] * Price) / COUNT(*), 2) AS promedio_por_producto
FROM inventario
WHERE (1.0 * [Number of products sold] / [Stock levels]) < 2
GROUP BY [Product type]
ORDER BY valor_inmovilizados_categoria DESC
'''

#Exportación del Valor del stock inmovilizado a un csv limpio 
df3 = pd.read_sql_query(query, conn)
df3.to_csv('../data/processed/valor_inmovilizados_categoria.csv', index=False)

In [64]:
# Calculo del porcentaje del stock inmovilizado
valor_total = df1['valor_total_stock'][0]
valor_inmovilizado = df2['valor_stock_inmovilizado'][0]

porcentaje = (valor_inmovilizado / valor_total) * 100
df_porcentaje = pd.DataFrame({
    'porcentaje_inmovilizado': [round(porcentaje, 2)]})

#Exportación del Valor del stock inmovilizado a un csv limpio
df_porcentaje.to_csv('../data/processed/porcentaje_inmovilizado.csv', index=False)

In [65]:
conn.close()